In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_auto_sys as aus
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Baseline_node2vec
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_AS_ablation_n2v.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.01},
 'model_path': 'models/AS_Ablation_N2V/',
 'classifier_hidden_size': 20,
 'ffn_fusion_size': 10,
 'ffn_hidden_size': 10,
 'gcn_fusion_size': 20,
 'spatial_hidden_size': 20,
 'spatial_input_dim': 128,
 'temporal_hidden_size': 20,
 'temporal_input_dim': 200,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/AS-733/',
 'prep_data_path': 'prep_data/AS733_neg/'}

In [4]:
data = aus.Autonomous_System_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True, neg_sample=1000)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Baseline_node2vec(config['spatial_hidden_size'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])

Dataset splits sizes:  train 60 dev 10 test 20


In [5]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.38524464462029234
Validation Metric MAPs: 0.09913403963616528
Test Metric MRRs: 0.36073824824837714
Test Metric MAPs: 0.09476286372433218


(0.36073824824837714, 0.09476286372433218)